In [ ]:
import os

from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
import h5py

from gala.observation import apparent_magnitude
import gala.dynamics as gd
import gala.coordinates as gc
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('apw-notebook')
%matplotlib inline

from sklearn.neighbors import KDTree

In [ ]:
tgas_path = os.path.abspath("../data/tgas.hdf5")

In [ ]:
with h5py.File(tgas_path, 'r') as f:
    tgas = f['tgas'][:]
    tgas = tgas[(tgas['parallax'] / tgas['parallax_error']) > 16]
len(tgas)

In [ ]:
c = coord.ICRS(ra=tgas['ra']*u.deg, dec=tgas['dec']*u.deg,
               distance=(tgas['parallax']*u.mas).to(u.pc, equivalencies=u.parallax()))
# pm = np.vstack((clean_tgas_rave['pmra'],clean_tgas_rave['pmdec'])) * u.mas/u.yr
_ix = c.distance < 100*u.pc
local_tgas = tgas[_ix]

xyz = c.transform_to(coord.Galactocentric).cartesian.xyz[:,_ix]
# vxyz = gc.vhel_to_gal(c, pm=pm, rv=rv, vlsr=[0,0,0]*u.km/u.s)

# w0 = gd.CartesianPhaseSpacePosition(pos=xyz[:,_ix], vel=vxyz[:,_ix])
# print(w0.shape)

# xyz = w0.pos.to(u.kpc)
# vxyz = w0.vel.to(u.km/u.s) - np.median(w0.vel.to(u.km/u.s), axis=1)[:,None]
_ix.sum()

In [ ]:
X = xyz.to(u.pc).value.T
kd = KDTree(X)

In [ ]:
_d,_tree_i = kd.query(X, k=2)
d = _d[:,1]
tree_i = _tree_i[:,1]

In [ ]:
plt.hist(np.log10(d), bins=np.linspace(-3,2.,32));
plt.yscale('log')
plt.xlabel(r'$\log_{10}(d/{\rm pc})$')

In [ ]:
pm = np.vstack((local_tgas['pmra'], local_tgas['pmdec'])).T
dpm = (pm - pm[tree_i])

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(12,5),sharex=True,sharey=True)

axes[0].semilogx(d, dpm[:,0]/local_tgas['parallax']*4.74, ls='none', marker='.', alpha=0.2)
axes[1].semilogx(d, dpm[:,1]/local_tgas['parallax']*4.74, ls='none', marker='.', alpha=0.2)

axes[0].set_xlabel(r'$d$ [pc]')
axes[1].set_xlabel(r'$d$ [pc]')

axes[0].set_ylabel(r'$\Delta \mu_\alpha/\varpi$ [km/s]')
axes[1].set_ylabel(r'$\Delta \mu_\delta/\varpi$ [km/s]')

axes[0].set_xlim(5E-3,2.5)
axes[0].set_ylim(-10,10)

axes[0].axhline(0, alpha=0.1)
axes[1].axhline(0, alpha=0.1)

fig.tight_layout()

$$
\begin{align}
x &= \mu_\alpha \, \cos\delta\\
y &= \mu_\delta\\
z &= 4.74^2\frac{(x_2-x_1)^2 + (y_2-y_1)^2}{\varpi_1^2}
\end{align}
$$

$$
a = x_2 - x_1\\
b = y_2 - y_1\\
z = A \, \frac{a^2 + b^2}{\varpi^2}\\
$$

$$
\sigma_z = A [ \frac{2a}{\varpi^2}\sigma_a + \frac{2b}{\varpi^2}\sigma_b - 2\frac{a^2+b^2}{\varpi^3}\sigma_\varpi]
$$

In [ ]:
dpm_mag = np.sqrt(np.sum(dpm**2, axis=1))

a = dpm[:,0]
b = dpm[:,1]
da = np.sqrt(local_tgas['pmra']**2 + local_tgas['pmra'][tree_i]**2)
db = np.sqrt(local_tgas['pmdec']**2 + local_tgas['pmdec'][tree_i]**2)
pom = local_tgas['parallax']
dpom = local_tgas['parallax_error']
yerr = 4.74*(2*a/pom**2*da + 2*b/pom**2*db - 2*(a**2+b**2)/pom**3*dpom)

fig,ax = plt.subplots(1,1,figsize=(8,6))

ax.errorbar(d, dpm_mag/local_tgas['parallax']*4.74, yerr=yerr, ls='none', marker='.', alpha=0.2)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel(r'$d$ [pc]')
ax.set_ylabel(r'$||\Delta \mu||/\varpi$ [km/s]')

ax.set_xlim(5E-3,2.5)
ax.set_ylim(-0.25,10)
ax.axhline(0, alpha=0.1)

fig.tight_layout()

In [ ]:
from astropy.constants import G
np.sqrt(G * 2*u.Msun / (1*u.pc)).to(u.km/u.s)